In [ ]:
import numpy as np
import sys
import time
from dask.distributed import as_completed

In [ ]:
class Model:
    def __init__(
        self,
        bc=0,
        courant=0.3,
    ):
        self.bc = bc
        self.courant = courant

In [ ]:
def process_one_shot(model, dm_in):
    time.sleep(60)
    imgglobal=np.ones([100, 100, 100])
    return imgglobal

def process_many_shots_using_dask(client, model, dm_in):
    imgglobal=np.zeros([100, 100, 100])
    dm_in_future = client.scatter(dm_in)
    nshots = 50
    
    futures = []
    for i in range(0, nshots):
        futures.append(client.submit(process_one_shot, model, dm_in_future))
    seq = as_completed(futures)
    while seq.count() > 1:
        a = next(seq)
        b = next(seq)
        new = client.submit(np.add, a, b)
        seq.add(new)

    imgglobal = np.add(imgglobal, next(seq).result())
    imgglobal /= nshots
    return imgglobal

In [ ]:
# from dask.distributed import Client, LocalCluster
# # cluster = LocalCluster(n_workers = 4, threads_per_worker = 1) # works - first round every worker gets at least 1 task
# cluster = LocalCluster(n_workers = 4) # not work - first round only one worker gets 50 tasks
# client = Client(cluster)
# client

In [ ]:
import env_modules_python as lmod
lmod.module('load', 'sge')
from dask_jobqueue import SGECluster
from dask.distributed import Client
cluster = SGECluster(
    nanny = False,
    processes = 1,  # number of workers per job
    cores = 1,      # make sure nthreads == 1, each dask worker forks one thread
    scheduler_options = dict(dashboard_address=':0'),
    job_extra_directives = ['-pe {} {}'.format('sapphirerapids', 112),
                            '-cwd',
                            '-S /bin/csh',
                            '-j y',
                            '-o /dev/null'],
    job_script_prologue=['module load conda', 
                         'conda activate intel-2021', 
                         f'setenv OMP_NUM_THREADS {24}'],
    memory = '600 GiB',
    project = 'hpcteam',
    walltime = '00:29:58'
)
client = Client(cluster)
# when scale to 10 workers/nodes, every worker gets at least one task in the first round;
# when scale to 25 workers/nodes, only one worker gets all tasks in the first round
cluster.scale(25) 
client.wait_for_workers(25)
client

In [ ]:
model = Model(30, courant = 0.35)
nxmodel = 1271
nymodel = 1299
nzmodel = 675
dm_in = np.zeros([nxmodel,nymodel,nzmodel])
image_up_dev = np.zeros([nxmodel,nymodel,nzmodel]) 
niter=10
for k in range(niter):
    dm_in= image_up_dev
    print('Iteration',k+1)
    imgglobal = process_many_shots_using_dask(client, model, dm_in)